# 첫 번째 신경망 훈련하기 : 기초적인 분류문제

In [ ]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

load_data() 함수를 호출하면 네 개의 넘파이(NumPy) 배열이 반환됨

> train_images와 train_labels 배열은 모델 학습에 사용되는 훈련 세트

> test_images와 test_labels 배열은 모델 테스트에 사용되는 테스트 세트

이미지는 28x28 크기의 넘파이 배열이고 픽셀 값은 0과 255 사이 레이블(label)은 0에서 9까지의 정수 배열. 이 값은 이미지에 있는 옷의 클래스(class)를 나타내냄

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 데이터 탐색

훈련하기 전 데이터셋 구조 탐색


In [ ]:
train_images.shape

## 데이터 전처리

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

이 이미지의 픽셀값 범위는 0~255이다.

 이것을 0~1 사이로 조정한다. 이때 훈련 데이터와 테스트 세트를 동일하게 처리한다.

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## 모델 구성

신경망 모델을 만들려면 모델의 층을 구성한 다음 모델을 컴파일 한다.

## 층설정

신경망의 기본 구성 요소는 layer이다. 각 layer은 주입된 데이터에서 표현(feature)를 추출한다. 아마 문제를 해결하는데 더 의미있는 feature가 추출될 것이다.

대부분은 딥러닝은 간단한 층을 연결하여 구성된다. 

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

tf.keras.layers.Flatten은 2차원 배열을 28*28 = 784의 1차원 배열로 변환한다. 이 층은 이미지에 있는 픽셀의 행을 펼쳐서 일렬로 늘린다. 이 층에 가중치는 없다. 

픽셀을 펼친 후 두개의 Dense가 연결된다. 첫번째 층은 128개의 노드를 갖는다. 두번째 층은 10개의 노드의 softmax 층이다. 이 층은 10개의 확률을 반환하고 반환된 값의 전체 합은 1이다. 각 노드는 현재 이미지가 10개의 클래스 중 하나에 속할 확률을 출력한다.

## 모델 컴파일

모델을 훈련하기 전에 볓가지 설정이 컴파일 단계에서 추가된다.

> 손실 함수 : 훈련 하는 동안 모델의 오차를 측정한다. 모델의 학습이 올바른 방향으로 향하도록 이 함수를 최소화 해야한다.

> 옵티마이저 : 데이터의 손실 함수를 바탕으로 모델의 업데이트 방법을 결정.

> 지표 : 훈련 단계와 테스트 단계를 모니터링하기 위해 사용된다. 

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 모델 훈련

신경망 모델을 훈련하는 단계는 다음과 같다.

1. 훈련 데이터를 모델에 주입. 
2. 모델이 이미지와 레이블을 매핑하는 방법을 배운다.
3. 테스트 세트에 대한 모델의 예측을 만든다.



In [ ]:
model.fit(train_images, train_labels, epochs=5)

## 정확도 평가

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\n테스트 정확도:', test_acc)

훈련 데이터는 0.98 테스트 데이터는 0.86으로 나왔다. 테스트의 정확도가 훈련 정확도보다 낮게 나온 것은 **과대적합**때문이다. 과대적합이란 머신러닝 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 현상을 말한다.

## 예측 만들기

훈련 모델을 사용하여 이미지에 대한 예측을 만든다.

In [ ]:
predictions = model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [ ]:
# 처음 X 개의 테스트 이미지와 예측 레이블, 진짜 레이블을 출력합니다
# 올바른 예측은 파랑색으로 잘못된 예측은 빨강색으로 나타냅니다
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

In [ ]:
# 이미지 하나만 사용할 때도 배치에 추가합니다
img = test_images[0]

img = (np.expand_dims(img,0))
predictions_single = model.predict(img)

plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)